<h1 align="center"><img align="center" src="data/geoparse_logo.png" alt="GeoParse Logo" width="200"/></h1>
<h1 align="center">GeoParse</h1>
<h3 align="center">It's all about points <img src="data/point.png" width="10"/> lines <img src="data/line.png" width="40"/> and polygons <img src="data/polygon.png" width="30"/></h3>


#### [Website](http://geoparse.io) 
***

# Visualization

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/geoparse/geoparse/main?labpath=tutorials%2F00_visualization.ipynb)

This notebook demonstrates how to display:

* Points, lines, and polygons.
* Their heatmaps and clusters.
* Their coverage areas using geospatial cells such as H3, S2, and Geohash.
* Geospatial cells on a map based on their indexes.
* OSM roads and buildings using their IDs.
***

In [1]:
import warnings

import geopandas as gpd
import pandas as pd

from geoparse.karta import plp

warnings.filterwarnings("ignore")

## Read from CSV file

In [2]:
df = pd.read_csv("data/great_britain_road_casualties-2023.csv")
df.head()

,date,time,lat,lon,number_of_vehicles,number_of_casualties,speed_limit
0,03/01/2023,19:12,51.356551,-0.097759,1,1,30
1,07/01/2023,10:05,51.593701,0.022379,1,1,30
2,14/01/2023,16:15,51.466689,-0.011289,1,1,20
3,15/01/2023,19:51,51.671577,-0.037543,1,1,30
4,16/01/2023,19:22,51.447944,0.117279,2,1,30


In [3]:
len(df)

1522

In [4]:
plp(df)

In [6]:
plp(df, heatmap=True, cluster=True)

In [ ]:
plp(df, point_color="speed_limit", point_popup={"Speed limit": "speed_limit"})

## Read a Polygon from Geo File

In [ ]:
gdf = gpd.read_file("data/london.geojson")
gdf

In [ ]:
plp(gdf)

In [ ]:
plp(gdf, centroid=True)

In [ ]:
plp(gdf, h3_res=8)

In [ ]:
plp(gdf, h3_res=11, compact=True)

In [ ]:
plp(gdf, s2_res=12)

In [ ]:
plp(gdf, s2_res=16, compact=True)

In [ ]:
plp(gdf, geohash_res=5)

In [ ]:
plp(gdf, geohash_res=7, compact=True)

# Mutiple data frames

In [ ]:
plp([df, gdf], h3_res=6)

# Geospatial Cells

In [ ]:
import h3

lat, lon = 41.87, -87.78


# Get the H3 index at resolution 6 for the central point
h3_index = h3.latlng_to_cell(lat, lon, 6)

# Get adjacent H3 cells including the central cell itself
h3_cells = h3.grid_disk(h3_index, 1)  # k_ring with radius 1 returns the central cell + neighbors
h3_cells

In [ ]:
plp(cells=list(h3_cells), cell_type="h3")

# OSM Ways

In [ ]:
plp(osm_ways=[335265936, 53820456, 1117218957], s2_res=22, compact=True)

In [ ]:
plp(osm_ways=[260909736, 54454413], h3_res=8)